In [1]:
import pandas as pd
import joblib


In [2]:
scaler = joblib.load("Saved_objects/scaler.pkl")
encoder = joblib.load("Saved_objects/encoder.pkl")
model = joblib.load("Saved_objects/best_model.pkl")


In [3]:
def categorize_bmi(bmi):
    if bmi < 18.5:
        return "Underweight"
    elif bmi < 25:
        return "Normal weight"
    elif bmi < 30:
        return "Overweight"
    else:
        return "Obese"


numeric_features = ["Pregnancies", "Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI", "DiabetesPedigreeFunction", "Age"]
categorical_features = ["BMI_category"]


In [4]:
def inference_pipeline(sample):
    sample_df = pd.DataFrame([sample])

    
    if "BMI_category" not in sample_df:
        sample_df["BMI_category"] = sample_df["BMI"].apply(categorize_bmi)

    
    sample_df[numeric_features] = scaler.transform(sample_df[numeric_features])

    
    encoded_sample = encoder.transform(sample_df[categorical_features])
    encoded_sample_df = pd.DataFrame(encoded_sample, columns=encoder.get_feature_names_out(categorical_features))

    
    sample_df = sample_df.reset_index(drop=True)
    sample_df = pd.concat([sample_df, encoded_sample_df], axis=1).drop(columns=categorical_features)

    
    prediction = model.predict(sample_df)
    return prediction[0]

In [5]:
test_sample = {
    "Pregnancies": 6,
    "Glucose": 148,
    "BloodPressure": 72,
    "SkinThickness": 35,
    "Insulin": 0,
    "BMI": 33.6,
    "DiabetesPedigreeFunction": 0.627,
    "Age": 50
}

test_sample["BMI_category"] = categorize_bmi(test_sample["BMI"])

predicted_class = inference_pipeline(test_sample)
print(f"Predicted Class: {predicted_class}")

Predicted Class: 1


In [6]:
val_data = pd.read_csv("Data/diabetes.csv")

val_data["BMI_category"] = val_data["BMI"].apply(categorize_bmi)

for i in range(5):
    sample = val_data.iloc[i].drop("Outcome").to_dict()
    predicted_class = inference_pipeline(sample)
    print(f"Sample {i + 1}: Predicted Class = {predicted_class}")


Sample 1: Predicted Class = 1
Sample 2: Predicted Class = 0
Sample 3: Predicted Class = 1
Sample 4: Predicted Class = 0
Sample 5: Predicted Class = 1
